# Ejercicio 1

In [20]:
import pandas as pd

class NaiveBayesClassifier:

    def train(self, df, alpha=1):
        grouped_classes = df.groupby('Nacionalidad')
        self.classes = df['Nacionalidad'].unique()
        self.attributes = df.columns[0: -1]
        self.class_probabilities = {}
        self.attribute_probabilities = {}
        total_len = len(df)

        for _class, grouped in grouped_classes:
            data_len = len(grouped)
            self.class_probabilities[_class] = (data_len + alpha) / (total_len + alpha * len(self.classes))
            self.attribute_probabilities[_class] = {}
            for attribute in df.columns[0:-1]:    # Skipping last column (Nacionalidad)
                self.attribute_probabilities[_class][attribute] = (grouped[attribute].sum() + alpha) / (data_len + alpha * len(self.attributes))
    
    def print_probabilities(self, values):
        prediction = {}
        for _class in self.classes:
            probability = self.class_probabilities[_class]
            for i,value in enumerate(values):
                if value == 1:
                    probability *= self.attribute_probabilities[_class][self.attributes[i]]
                else: 
                    probability *= 1 - self.attribute_probabilities[_class][self.attributes[i]] # 1 - P(A) = P(not A)
            prediction[_class] = probability
        for _class, value in prediction.items():
            print(_class, value / sum(prediction.values()))  # divide the probability by the sum of all probabilities to get the normalized probability


file = 'inputs/PreferenciasBritanicos.xlsx'
brits_df = pd.read_excel(file)
classifier = NaiveBayesClassifier()
classifier.train(brits_df)
# 1.B
classifier.print_probabilities([1, 0, 1, 1, 1])
print()
# 1.C
classifier.print_probabilities([0, 1, 1, 0, 1])

I 0.25260461382104393
E 0.7473953861789561

I 0.623508361311908
E 0.37649163868809205


# Ejercicio 2

## Paso 1: preprocesamiento del dataset
Primero necesitamos hacer un preprocesamiento de los datos. Para esto decidimos reducir el dataset a 4 categorías, preferencialmente que no guarden demasiada correlación entre ellas en cuanto a las palabras utilizadas y asegurarnos que el modelo logre diferenciarlas correctamente sin tener que lidiar con las otras categorías. Una vez que el objetivo sea cumplido agregaremos devuelta las categorías filtradas para utilizar el dataset completo.

Para el preprocesamiento de los titulos utilizaremos tecnicas utilizadas en NLP y las aplicaremos a los titulos, procedemos a la tokenización de las palabras en los titulos,  

In [3]:
import re
import nltk
import pandas as pd
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

def preprocess_text_spanish(text):
    # Convert to lowercase and remove punctuation
    text = re.sub(r'[^\w\s]', '', text.lower())
    
    # Tokenization
    tokens = text.split()
    
    # Remove stop words
    stop_words = set(stopwords.words("spanish"))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming (use SnowballStemmer for Spanish)
    stemmer = SnowballStemmer("spanish")
    tokens = [stemmer.stem(token) for token in tokens]
    
    return tokens

file = 'inputs/NoticiasArgentinas.xlsx'
news_df = pd.read_excel(file)
accepted_cats = ['Nacional', 'Deportes', 'Salud', 'Ciencia y Tecnologia']
filtered_df = news_df[news_df['categoria'].isin(accepted_cats)]
labels = filtered_df['categoria']

preprocessed_titles = [preprocess_text_spanish(title) for title in filtered_df['titular']] # a cada titular le aplico la función
preprocessed_titles_strings = [" ".join(tokens) for tokens in preprocessed_titles]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(preprocessed_titles_strings)

# Build the vocabulary
vocabulary = vectorizer.get_feature_names_out()
vocabulary_size = len(vocabulary)

class_frequency_map = labels.value_counts(normalize=True).to_dict() #calcula la frecuencia relativa para cada clase
word_freq_per_class = defaultdict(lambda: defaultdict(lambda: 0))
total_tokens_per_class = defaultdict(int)

# Count word occurrences in each class
for i, doc in enumerate(X):
    tokens = preprocessed_titles[i]
    total_tokens_per_class[labels.iloc[i]] += len(tokens)
    for word_idx in doc.indices:
        word = vocabulary[word_idx]
        for token in tokens:
            word_freq_per_class[token][labels.iloc[i]] += 1
            

# Normalize word frequency per class by dividing by total tokens in each class
for word in word_freq_per_class:
    for class_label in word_freq_per_class[word]:
        word_freq_per_class[word][class_label] /= total_tokens_per_class[class_label]
        
# Example new title to classify
new_title = "Paulo Gazzaniga, el arquero sorpresa de Lionel Scaloni"

# Preprocess the new title
new_title_tokens = preprocess_text_spanish(new_title)

# Calculate conditional probabilities
predicted_probabilities = {}
for class_label in class_frequency_map.keys():
    class_frequency = class_frequency_map[class_label]
    prob_word_given_class = class_frequency
    for token in new_title_tokens:
        word_freq = word_freq_per_class[token][class_label]
        if word_freq == 0:
            # Applying Laplace smoothing
            smoothed_prob = 1 / (total_tokens_per_class[class_label] + vocabulary_size)
            prob_word_given_class *= smoothed_prob
        else:
            prob_word_given_class *= word_freq / sum(word_freq_per_class[token].values())
    predicted_probabilities[class_label] =prob_word_given_class

# Predict the category with the highest probability
print(predicted_probabilities)
predicted_category = max(predicted_probabilities, key=predicted_probabilities.get)
print("Predicted Category:", predicted_category)



{'Nacional': 1.0328170954114763e-24, 'Ciencia y Tecnologia': 3.7549539696419673e-25, 'Deportes': 0.1865889585388724, 'Salud': 3.357792680081478e-28}
Predicted Category: Deportes
